In [ ]:
# Read CSV , Calculate Lead Time , Create Lead Time Labels and create Nomic Atlas Map

In [78]:
import pandas as pd
import dask.dataframe as dd
import re
from string import punctuation
from datetime import datetime
from nomic import atlas
import nomic
import random
import math

In [79]:
def remove_nan_rows_by_column(records, column_name):
    # Find the index of the specified column by name
    column_index = None
    if records:
        headers = records[0]
        if column_name in headers:
            column_index = headers.index(column_name)

    if column_index is None:
        raise ValueError(f"Column '{column_name}' not found in the records.")

    # Filter out rows where the column value is NaN
    return [row for row in records if not math.isnan(row[column_index])]


In [80]:
# Function to get difference between two dates
def date_difference(date_str1, date_str2):
    # Convert the date strings to datetime objects
    date1 = datetime.strptime(date_str1, "%Y-%m-%d %H:%M:%S")
    date2 = datetime.strptime(date_str2, "%Y-%m-%d %H:%M:%S")

    # Calculate the time difference
    time_difference = date2 - date1

    return time_difference

In [81]:
# Funtion to format timedelta to hour.minutes format
def format_to_hh_mm(timedelta_obj):
    # Calculating total seconds
    total_seconds = timedelta_obj.total_seconds()

    # Converting total seconds to hours and minutes
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60

    # Combining hours and minutes into a float
    hours_and_minutes = hours + (minutes / 60)

    return round(hours_and_minutes, 2)

In [51]:
all_data = pd.read_csv("Final_security_dataset.csv" , index_col=0)

In [82]:
data = all_data[:50000]
#Get only records which has duplicate title
duplicates = data[data.duplicated('title', keep=False)]
len(duplicates)

27828

In [83]:
def generate_lead_time(matching_rows):
    
        closed_ticket = matching_rows[matching_rows['action'] == 'closed']
        open_ticket = matching_rows[matching_rows['action'] == 'opened']
        
        if not closed_ticket.empty and not open_ticket.empty:
        #and closed_ticket['repo_url'].values[0] == open_ticket['repo_url'].values[0]:
        
            close_time = closed_ticket['created_at'].values[0]
            open_time = open_ticket['created_at'].values[0]
           
            time_diff = date_difference(close_time,open_time)
            lead_time = format_to_hh_mm(time_diff)

            return lead_time
        else:
            return 0 # Put N/A (Not Available) if one of open_ticket or close_ticket is empty

In [84]:
visited_titles = set()

duplicates_with_lead_time = duplicates.copy()

# Fills duplicates_with_lead_time with same title sharing same repo
for i in range(len(duplicates)):
    row_title = duplicates.iloc[i]['title']   
        
    if row_title not in visited_titles:
        # Process the duplicate row
        
        matching_titles = duplicates[duplicates['title'] == row_title]     
        
        matching_titles_with_matching_repo = matching_titles[matching_titles.duplicated('repo', keep=False)]
        
        # this means there are both open and close record for same title and same repo
        if len(matching_titles_with_matching_repo) > 0:
        
            lead_time = generate_lead_time(matching_titles_with_matching_repo)

            condition = (duplicates['title'] == row_title) & (duplicates['repo'] ==  matching_titles_with_matching_repo['repo'].values[0])

            duplicates_with_lead_time.loc[condition, 'lead_time'] = abs(lead_time)
      
            
        # Mark the row as visited
        visited_titles.add(row_title)

In [89]:
#remove records with lead_time NaN
filtered_duplicates = duplicates_with_lead_time.dropna(subset=['lead_time'])

len(filtered_duplicates)

14021

In [90]:
#remove records with lead_time 0.00

filtered_duplicates = filtered_duplicates[filtered_duplicates['lead_time'] != 0.00]
len(filtered_duplicates)

11242

In [91]:
sorted_duplicates = filtered_duplicates.sort_values('lead_time').reset_index(drop=True)

len(sorted_duplicates)

11242

In [92]:
total_rows = len(sorted_records)
first_index = total_rows // 4
second_index = first_index * 2
third_index = first_index * 3


In [93]:
# Category A = Lowest quarter of lead time ... Category D = Highest quarter of lead time
sorted_records['Category'] = 'Longest_Lead_Time'  # Default to Category D
sorted_records.loc[:first_index, 'Category'] = 'Shortest_Lead_Time'
sorted_records.loc[first_index:second_index, 'Category'] = 'Moderate_Short_Lead_Time'
sorted_records.loc[second_index:third_index, 'Category'] = 'Moderate_Long_Lead_Time'

In [94]:
sorted_records[:10]

,id,type,created_at,repo,repo_url,action,title,labels,body,index,lead_time,Category,body_text,text
0,2.490469e+09,IssuesEvent,2015-01-02 15:34:19,izuzero/xe-module-ajaxboard,https://api.github.com/repos/izuzero/xe-module...,opened,SSE Listener session stack 제한,type/enhancement type/security,"SSE Listener session은 현재 만료일을 30분으로 지정해두고 있으나,...",True,0.0,Shortest_Lead_Time,"SSE Listener session은 현재 만료일을 30분으로 지정해두고 있으나,...",sse listener session은 현재 만료일을 30분으로 지정해두고 있으나 ...
1,8.665837e+09,IssuesEvent,2018-11-29 01:08:20,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2018-3774 High Severity Vulnerability dete...,security vulnerability,## CVE-2018-3774 - High Severity Vulnerability...,True,0.0,Shortest_Lead_Time,## CVE-2018-3774 - High Severity Vulnerability...,cve20183774 high severity vulnerability detail...
2,8.665836e+09,IssuesEvent,2018-11-29 01:07:59,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2016-1000340 High Severity Vulnerability d...,security vulnerability,## CVE-2016-1000340 - High Severity Vulnerabil...,True,0.0,Shortest_Lead_Time,## CVE-2016-1000340 - High Severity Vulnerabil...,cve20161000340 high severity vulnerability det...
3,8.665832e+09,IssuesEvent,2018-11-29 01:06:53,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2018-11697 High Severity Vulnerability det...,security vulnerability,## CVE-2018-11697 - High Severity Vulnerabilit...,True,0.0,Shortest_Lead_Time,## CVE-2018-11697 - High Severity Vulnerabilit...,cve201811697 high severity vulnerability detai...
4,8.665831e+09,IssuesEvent,2018-11-29 01:06:38,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2016-10531 Medium Severity Vulnerability d...,security vulnerability,## CVE-2016-10531 - Medium Severity Vulnerabil...,True,0.0,Shortest_Lead_Time,## CVE-2016-10531 - Medium Severity Vulnerabil...,cve201610531 medium severity vulnerability det...
5,8.665824e+09,IssuesEvent,2018-11-29 01:04:46,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,WS-2015-0024 High Severity Vulnerability detec...,security vulnerability,## WS-2015-0024 - High Severity Vulnerability\...,True,0.0,Shortest_Lead_Time,## WS-2015-0024 - High Severity Vulnerability\...,ws20150024 high severity vulnerability details...
6,8.665816e+09,IssuesEvent,2018-11-29 01:02:38,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2018-3774 High Severity Vulnerability dete...,security vulnerability,## CVE-2018-3774 - High Severity Vulnerability...,True,0.0,Shortest_Lead_Time,## CVE-2018-3774 - High Severity Vulnerability...,cve20183774 high severity vulnerability detail...
7,8.665837e+09,IssuesEvent,2018-11-29 01:08:24,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2017-12597 High Severity Vulnerability det...,security vulnerability,## CVE-2017-12597 - High Severity Vulnerabilit...,True,0.0,Shortest_Lead_Time,## CVE-2017-12597 - High Severity Vulnerabilit...,cve201712597 high severity vulnerability detai...
8,8.665813e+09,IssuesEvent,2018-11-29 01:01:37,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2016-1000340 High Severity Vulnerability d...,security vulnerability,## CVE-2016-1000340 - High Severity Vulnerabil...,True,0.0,Shortest_Lead_Time,## CVE-2016-1000340 - High Severity Vulnerabil...,cve20161000340 high severity vulnerability det...
9,8.658768e+09,IssuesEvent,2018-11-28 02:34:23,ekirmayer/ExerciseFiles,https://api.github.com/repos/ekirmayer/Exercis...,opened,CVE-2018-11039 Medium Severity Vulnerability d...,security vulnerability,## CVE-2018-11039 - Medium Severity Vulnerabil...,True,0.0,Shortest_Lead_Time,## CVE-2018-11039 - Medium Severity Vulnerabil...,cve201811039 medium severity vulnerability det...


In [95]:

sorted_records["body_text"] = sorted_records['body'].astype(str)


In [96]:
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    text = re.sub(r"https?://\S+", "", text)
    return text

sorted_records["text"] = sorted_records["body_text"].map(preprocess_text)

In [97]:
clean_df = sorted_records[["title" , "text", "lead_time","Category"]]
documents = clean_df.to_dict(orient='records')


In [98]:
for i in range(len(documents)):
    documents[i]['id'] = i

In [100]:
project = atlas.map_text(data=documents,
                         id_field='id',
                         indexed_field='text',
                         colorable_fields=['title','Category'],
                         name='Security_data_lead_time_map', build_topic_model=True,
                         description='Security_data_lead_time'
                         )

2024-01-07 19:34:16.775 | INFO     | nomic.project:_create_project:790 - Creating project `Security_data_lead_time_map` in organization `pundarika.click`
2024-01-07 19:34:17.767 | INFO     | nomic.atlas:map_text:248 - Uploading text to Atlas.
2024-01-07 19:34:17.808 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32
3it [00:01,  1.66it/s]                                                          
2024-01-07 19:34:19.614 | INFO     | nomic.project:_add_data:1422 - Upload succeeded.
2024-01-07 19:34:19.619 | INFO     | nomic.atlas:map_text:284 - Text upload succeeded.
2024-01-07 19:34:21.136 | INFO     | nomic.project:create_index:1132 - Created map `Security_data_lead_time_map` in project `Security_data_lead_time_map`: https://atlas.nomic.ai/map/f02e6512-24a6-4ba0-9985-09bc9ffe2da1/5456d2be-01a1-40c5-b1fa-b288e059bdd8
2024-01-07 19:34:21.140 | INFO     | nomic.atlas:map_text:300 - Security_data_lead_time_map: http